# Goals: 
In this notebook I am attempting to do the following: 

1. Set up an SQL table with sql alchemy. 
2. Use the the Beautiful soup parsers that I have written to parse the html and then place into a database. 

I had to remake one of th tables. 

### First import the libraries 
    

In [1]:
# modules
%matplotlib inline
%config InlineBackend.figure_format='retina'   #this makes it retina style (oh yeah!)
from bs4 import BeautifulSoup
from os.path import join
from pandas import DataFrame
from os import getcwd, listdir
import matplotlib.pyplot as plt
import pandas as pd


#append the path for my own funtions and import them 
import sys
sys.path.append('/home/ubuntu/InsightProject')
from scraper import * 
from data_functions import *


from multiprocessing.pool import ThreadPool as Pool
from validators.url import url as validate_url

In [2]:
import sqlalchemy as sa
from sqlalchemy import Column, Table, MetaData,create_engine
from sqlalchemy.dialects.mysql import *
from sqlalchemy.sql.expression import insert

### Use sqlalchemy to establish a connection to the database. 
In sql server I created a database called url. I also had to install and configure the SQL database. I am now going to use this sql data base to contain my strings. 

In [3]:
engine = sa.create_engine('mysql://root:@localhost/url')

In [4]:
results= engine.execute('show databases').fetchall()

In [5]:
results

[('information_schema',), ('mysql',), ('performance_schema',), ('url',)]

### Create a table to hold the data for the scrapped websites
1. First create a metadata object and bind it to the engine.
2. Then make the table. This table hold an ID, a url (the string), and a medium text 

In [6]:
metadata =MetaData()
metadata.bind = engine

I used the following commands to create the tables

```python

url =Table('classifier_htmls',metadata,
            Column('urlID', INTEGER, primary_key=True, autoincrement=True),
            Column('url', VARCHAR(60)),
            Column('htmlsText',MEDIUMTEXT),
            Column('htmlsSize', INTEGER))


student = Table('student_htmls',metadata,
            Column('urlID', INTEGER, primary_key=True, autoincrement=True),
            Column('url', VARCHAR(60)),
            Column('htmlsText',MEDIUMTEXT),
            Column('htmlsSize', INTEGER))
            
metadata.create_all(checkfirst=True)

```

In [8]:
engine.execute('show tables').fetchall()

[('classifier_htmls',), ('student_htmls',)]

In [9]:
engine.execute('describe student_htmls').fetchall()

[('urlID', 'int(11)', 'NO', 'PRI', None, 'auto_increment'),
 ('url', 'varchar(60)', 'YES', '', None, ''),
 ('htmlsText', 'mediumtext', 'YES', '', None, ''),
 ('htmlsSize', 'int(11)', 'YES', '', None, '')]

## Now time to use the beautiful soup functions to start parsing. 

First I will write a function to be called by the multiprocessing pools. 
This function will: 

1. Import the text, and scrape it. 
2. It will calculate the number of strings for each text. 
3. It will then insert the url, text, and the length of the string into the text. 

In [10]:
eng = create_engine('mysql://root:@localhost/url',pool_size=32)   #create a new engine with 32 connections

In [11]:
meta = MetaData()
meta.bind = eng

In [26]:
classifier_htmls = Table('classifier_htmls',metadata,
            Column('urlID', INTEGER, primary_key=True, autoincrement=True),
            Column('url', VARCHAR(60)),
            Column('htmlsText',MEDIUMTEXT),
            Column('htmlsSize', INTEGER))

InvalidRequestError: Table 'classifier_htmls' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [17]:
#``` python
def parse_text(html):
    """ parses the html and inserts into the database"""
    
    # parste the text and calculate the size
    html_path = '/home/ubuntu/data/html'
    url = html   #get the urginal url
    html = join(html_path, html)
    parsed = scrape_all_text(html)
    size = len(parsed)
    
    # handle the insertion into the table. 
    eng.dispose()
    with eng.connect() as conn:
        # insert the data into the database. 
        ins = classifier_htmls.insert().values(url = url, htmlsText = parsed, htmlsSize=size)
        conn.execute(ins)
#    ```

In [18]:
html_path = '/home/ubuntu/data/html'
htmls= [html for html in listdir('/home/ubuntu/data/html')]

In [23]:
print 'the number of entries is %d' %len(eng.execute('select * from classifier_htmls').fetchall())

the number of entries is 0


``` python i = parse_text(htmls[0]) ```

In [24]:
#I used this script to parse the data. I am going to write a new fuction to scrape t

#```python

def insert_htmls(htmls,threads = 32):
    """inserts multoples"""
    pool = Pool(threads)
    pool.map(parse_text, htmls)
    pool.close()   #```

In [25]:
#```python 
insert_htmls(htmls) #```

NameError: global name 'classifier_htmls' is not defined

### Add more the data that I previously downloaded to the data base. 

The goal of this part is to repeat what I did, scrape the remaing sites and input them into the sql database.  To do this. 

1. Set the path and file location. 
2. Modify the two fuctions. 
3. Run it. 

In [35]:
htmls= [html for html in listdir('/home/ubuntu/data/GoGuardianHTMLS-9-17-2015-A')]

In [36]:
def parse_text(html):
    """ parses the html and inserts into the database"""
    
    # parste the text and calculate the size
    html_path = '/home/ubuntu/data/GoGuardianHTMLS-9-17-2015-A'
    url = html   #get the urginal url
    html = join(html_path, html)
    parsed = scrape_all_text(html)
    size = len(parsed)
    
    # handle the insertion into the table. 
    eng.dispose()
    with eng.connect() as conn:
        # insert the data into the database. 
        ins = classifier_htmls.insert().values(url = url, htmlsText = parsed, htmlsSize=size)
        conn.execute(ins)

In [37]:
def insert_htmls(htmls,threads = 32):
    """inserts multoples"""
    pool = Pool(threads)
    pool.map(parse_text, htmls)
    pool.close() 

```insert_htmls(htmls)```

In [64]:
r = engine.execute('Select * from classifier_htmls').fetchall()

In [65]:
len(r)

30643

In [68]:
path1 = '/home/ubuntu/data/html'
path2 = '/home/ubuntu/data/GoGuardianHTMLS-9-17-2015-A'
len(listdir(path1)) +len(listdir(path2))

53540

In [72]:
data = pd.read_sql('Select * from classifier_htmls', con = engine)

In [73]:
data

,urlID,url,htmlsText,htmlsSize
0,1,thesinglesdatingagency-uk_co_uk.txt,Singles Dating Agency Register for free ...,55
1,2,faqt_net.txt,,5
2,3,stylenik_com.txt,403 Forbidden,18
3,4,lawschoolpoker_com.txt,You don t have permission to access on this ...,183
4,5,pat-raceable_tk.txt,pat raceable tk,20
5,6,media4_sitebrand_com.txt,,6
6,7,firearmstraininggroup_com.txt,FIREARMSTRAININGGROUP COM,30
7,8,mystmanuualfastweb_tk.txt,mystmanuualfastweb tk,26
8,9,moviefox_org.txt,An appropriate representation of the requested...,227
9,10,hitspages_com.txt,hitspages com Buy this domain Privacy Poli...,51
